In [1]:

import numpy as np
import pandas as pd

import lightgbm as lgb

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
#读取数据
age_train = pd.read_csv("age_train.csv", names=['uid','age_group'])
age_test = pd.read_csv("age_test.csv", names=['uid'])
user_basic_info = pd.read_csv("user_basic_info.csv", names=['uid','gender','city','prodName','ramCapacity','ramLeftRation','romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
user_behavior_info = pd.read_csv("user_behavior_info.csv", names=['uid','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum'])
user_app_actived = pd.read_csv("user_app_actived.csv", names=['uid','appId'])
#user_app_usage = pd.read_csv("user_app_usage.csv")
app_info = pd.read_csv("app_info.csv", names=['appId', 'category'])


In [4]:
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def f(x):
    s = x.value_counts()
    return np.nan if len(s) == 0 else s.index[0]
def processUserAppUsage():
    resTable = pd.DataFrame()
    reader = pd.read_csv("user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
    last_df = pd.DataFrame()
    
    app_info = pd.read_csv("app_info.csv", names=['appId','category'])
    cats = list(set(app_info['category']))
    category2id = dict(zip(sorted(cats), range(0,len(cats))))
    id2category = dict(zip(range(0,len(cats)), sorted(cats)))
    app_info['category'] = app_info['category'].apply(lambda x: category2id[x])
    i = 1
    
    while True:
        try:
            print("index: {}".format(i))
            i+=1
            df = reader.get_chunk(1000000)
            df = pd.concat([last_df, df])
            idx = df.shape[0]-1
            last_user = df.iat[idx,0]
            while(df.iat[idx,0]==last_user):
                idx-=1
            last_df = df[idx+1:]
            df = df[:idx+1]

            now_df = pd.DataFrame()
            now_df['uid'] = df['uid'].unique()
           
            
            tmp = df.groupby('uid')['appId'].count().to_frame().reset_index()
           
            now_df = now_df.merge(tmp, how='left', on='uid')
         
            now_df = now_df.merge(df.groupby('uid')['appId','use_date'].agg(['nunique']).reset_index(), how='left', on='uid')
          
            now_df = now_df.merge(df.groupby('uid')['duration','times'].agg(['mean','max','std']).reset_index(), how='left', on='uid')    
          
            now_df.columns = ['uid','usage_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std']
    
            df = df.merge(app_info, how='left', on='appId')
         
            now_df = now_df.merge(df.groupby('uid')['category'].nunique().to_frame().reset_index(), how='left', on='uid')
           
            now_df['usage_most_used_category'] = df.groupby(['uid'])['category'].transform(f)
            resTable = pd.concat([resTable, now_df])
        except StopIteration:
            break
    
    resTable.to_csv("appuseProcessed.csv",index=0)
    
    print("Iterator is stopped")

In [5]:
processUserAppUsage()

index: 1


d:\python27\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
d:\python27\lib\site-packages\pandas\core\generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27
index: 28
index: 29
index: 30
index: 31
index: 32
index: 33
index: 34
index: 35
index: 36
index: 37
index: 38
index: 39
index: 40
index: 41
index: 42
index: 43
index: 44
index: 45
index: 46
index: 47
index: 48
index: 49
index: 50
index: 51
index: 52
index: 53
index: 54
index: 55
index: 56
index: 57
index: 58
index: 59
index: 60
index: 61
index: 62
index: 63
index: 64
index: 65
index: 66
index: 67
index: 68
index: 69
index: 70
index: 71
index: 72
index: 73
index: 74
index: 75
index: 76
index: 77
index: 78
index: 79
index: 80
index: 81
index: 82
index: 83
index: 84
index: 85
index: 86
index: 87
index: 88
index: 89
index: 90
index: 91
index: 92
index: 93
index: 94
index: 95
index: 96
index: 97
index: 98
index: 99
index: 100
index: 101
index:

In [4]:
#将user_basic_info.csv 和 user_behavior_info.csv中的字符值编码成可以训练的数值类型，合并
class2id = {}
id2class = {}
def mergeBasicTables(baseTable):
    resTable = baseTable.merge(user_basic_info, how='left', on='uid', suffixes=('_base0', '_ubaf'))
    resTable = resTable.merge(user_behavior_info, how='left', on='uid', suffixes=('_base1', '_ubef'))
    cat_columns = ['city','prodName','color','carrier','os','ct']
    for c in cat_columns:
        resTable[c] = resTable[c].apply(lambda x: x if type(x)==str else str(x))
        sort_temp = sorted(list(set(resTable[c])))  
        class2id[c+'2id'] = dict(zip(sort_temp, range(1, len(sort_temp)+1)))
        id2class['id2'+c] = dict(zip(range(1,len(sort_temp)+1), sort_temp))
        resTable[c] = resTable[c].apply(lambda x: class2id[c+'2id'][x])
        
    return resTable

In [18]:
#处理app使用相关数据
#对user_app_actived.csv简单统计
#将之前训练的appuseProcess.csv进行合并
def mergeAppData(baseTable):
    resTable = baseTable.merge(user_app_actived, how='left', on='uid')
    resTable['appId'] = resTable['appId'].apply(lambda x: len(list(x.split('#'))))
    appusedTable = pd.read_csv("appuseProcessed.csv")
    resTable = resTable.merge(appusedTable, how='left', on='uid')
    resTable[['category', 'usage_most_used_category']] = resTable[['category', 'usage_most_used_category']].fillna(41)
    uadl = pd.read_csv("user_app_detail_list.csv")
    #print(uadl[:5])
    resTable = resTable.merge(uadl,how='left',on='uid')
    resTable = resTable.fillna(0)
    print(resTable[:5])
    return resTable

In [19]:
#合并用户基本特征以及app使用相关特征，作为训练集和测试集
df_train = mergeAppData(mergeBasicTables(age_train))
df_test = mergeAppData(mergeBasicTables(age_test))
print(df_train.shape)
print(df_test.shape)

       uid  age_group  gender  city  prodName  ramCapacity  ramLeftRation  \
0  1000001          4       0    51        78          3.0           0.43   
1  1000011          3       0    30       138          0.0           0.00   
2  1000015          5       1   228        78          3.0           0.34   
3  1000019          3       0    57       166          2.0           0.00   
4  1000023          2       1   293       164          2.0           0.34   

   romCapacity  romLeftRation  color  fontSize  ct  carrier  os  bootTimes  \
0         32.0           0.46     80      1.15   5        1  14        108   
1          0.0           0.00     16      0.00   6        1  15          0   
2         32.0           0.06     80      1.30   8        2  14         12   
3         17.0           0.00    100      0.00   7        3  15          0   
4         16.0           0.06    119      1.00   8        2  12          5   

   AFuncTimes  BFuncTimes  CFuncTimes  DFuncTimes  EFuncTimes  FFunc

In [20]:
#训练模型

from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [21]:
print("训练模型：")
param = {
        'learning_rate': 0.1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'max_depth': 20,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 230,
        'feature_fraction': 0.8,
        'metric': 'multi_error'
        }

X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

xx_score = []
cv_pred = []
skf = StratifiedKFold(n_splits=3, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    model = lgb.train(param, train, num_boost_round=1000, valid_sets=[vali], early_stopping_rounds=50)
    xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
    xx_pred = [np.argmax(x) for x in xx_pred]
    xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
    y_test = model.predict(test,num_iteration=model.best_iteration)
    y_test = [np.argmax(x) for x in y_test]
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission.csv',index=False)

训练模型：
0
training start...
[1]	valid_0's multi_error: 0.701493
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_error: 0.700209
[3]	valid_0's multi_error: 0.667954
[4]	valid_0's multi_error: 0.652565
[5]	valid_0's multi_error: 0.640553
[6]	valid_0's multi_error: 0.628517
[7]	valid_0's multi_error: 0.61432
[8]	valid_0's multi_error: 0.602795
[9]	valid_0's multi_error: 0.596407
[10]	valid_0's multi_error: 0.590417
[11]	valid_0's multi_error: 0.584683
[12]	valid_0's multi_error: 0.580498
[13]	valid_0's multi_error: 0.577332
[14]	valid_0's multi_error: 0.573717
[15]	valid_0's multi_error: 0.570662
[16]	valid_0's multi_error: 0.568332
[17]	valid_0's multi_error: 0.566141
[18]	valid_0's multi_error: 0.564131
[19]	valid_0's multi_error: 0.562435
[20]	valid_0's multi_error: 0.560735
[21]	valid_0's multi_error: 0.559071
[22]	valid_0's multi_error: 0.557531
[23]	valid_0's multi_error: 0.556165
[24]	valid_0's multi_error: 0.554865
[25]	valid_0's multi_error: 0.5534

[218]	valid_0's multi_error: 0.497263
[219]	valid_0's multi_error: 0.497234
[220]	valid_0's multi_error: 0.497238
[221]	valid_0's multi_error: 0.497034
[222]	valid_0's multi_error: 0.497023
[223]	valid_0's multi_error: 0.496892
[224]	valid_0's multi_error: 0.496771
[225]	valid_0's multi_error: 0.496672
[226]	valid_0's multi_error: 0.496684
[227]	valid_0's multi_error: 0.496687
[228]	valid_0's multi_error: 0.496622
[229]	valid_0's multi_error: 0.496493
[230]	valid_0's multi_error: 0.49641
[231]	valid_0's multi_error: 0.496387
[232]	valid_0's multi_error: 0.49632
[233]	valid_0's multi_error: 0.496254
[234]	valid_0's multi_error: 0.496143
[235]	valid_0's multi_error: 0.49604
[236]	valid_0's multi_error: 0.495908
[237]	valid_0's multi_error: 0.495907
[238]	valid_0's multi_error: 0.495862
[239]	valid_0's multi_error: 0.495781
[240]	valid_0's multi_error: 0.495741
[241]	valid_0's multi_error: 0.49565
[242]	valid_0's multi_error: 0.495571
[243]	valid_0's multi_error: 0.495535
[244]	valid_0's 

[435]	valid_0's multi_error: 0.488028
[436]	valid_0's multi_error: 0.487989
[437]	valid_0's multi_error: 0.487963
[438]	valid_0's multi_error: 0.488022
[439]	valid_0's multi_error: 0.488017
[440]	valid_0's multi_error: 0.487934
[441]	valid_0's multi_error: 0.487949
[442]	valid_0's multi_error: 0.487938
[443]	valid_0's multi_error: 0.487899
[444]	valid_0's multi_error: 0.487874
[445]	valid_0's multi_error: 0.487857
[446]	valid_0's multi_error: 0.487777
[447]	valid_0's multi_error: 0.487753
[448]	valid_0's multi_error: 0.487696
[449]	valid_0's multi_error: 0.487678
[450]	valid_0's multi_error: 0.487717
[451]	valid_0's multi_error: 0.48769
[452]	valid_0's multi_error: 0.487659
[453]	valid_0's multi_error: 0.487659
[454]	valid_0's multi_error: 0.487695
[455]	valid_0's multi_error: 0.487672
[456]	valid_0's multi_error: 0.487683
[457]	valid_0's multi_error: 0.487637
[458]	valid_0's multi_error: 0.487628
[459]	valid_0's multi_error: 0.487607
[460]	valid_0's multi_error: 0.487625
[461]	valid_0

[652]	valid_0's multi_error: 0.485022
[653]	valid_0's multi_error: 0.485032
[654]	valid_0's multi_error: 0.485038
[655]	valid_0's multi_error: 0.484996
[656]	valid_0's multi_error: 0.484983
[657]	valid_0's multi_error: 0.485007
[658]	valid_0's multi_error: 0.484996
[659]	valid_0's multi_error: 0.485011
[660]	valid_0's multi_error: 0.484971
[661]	valid_0's multi_error: 0.484949
[662]	valid_0's multi_error: 0.484928
[663]	valid_0's multi_error: 0.484917
[664]	valid_0's multi_error: 0.484893
[665]	valid_0's multi_error: 0.484904
[666]	valid_0's multi_error: 0.484923
[667]	valid_0's multi_error: 0.484904
[668]	valid_0's multi_error: 0.484865
[669]	valid_0's multi_error: 0.484878
[670]	valid_0's multi_error: 0.484871
[671]	valid_0's multi_error: 0.484852
[672]	valid_0's multi_error: 0.484798
[673]	valid_0's multi_error: 0.484811
[674]	valid_0's multi_error: 0.484825
[675]	valid_0's multi_error: 0.484807
[676]	valid_0's multi_error: 0.484793
[677]	valid_0's multi_error: 0.484769
[678]	valid_

[869]	valid_0's multi_error: 0.483468
[870]	valid_0's multi_error: 0.483466
[871]	valid_0's multi_error: 0.483446
[872]	valid_0's multi_error: 0.483459
[873]	valid_0's multi_error: 0.483459
[874]	valid_0's multi_error: 0.483407
[875]	valid_0's multi_error: 0.483395
[876]	valid_0's multi_error: 0.483389
[877]	valid_0's multi_error: 0.483398
[878]	valid_0's multi_error: 0.483386
[879]	valid_0's multi_error: 0.483392
[880]	valid_0's multi_error: 0.483429
[881]	valid_0's multi_error: 0.483426
[882]	valid_0's multi_error: 0.483441
[883]	valid_0's multi_error: 0.483374
[884]	valid_0's multi_error: 0.483401
[885]	valid_0's multi_error: 0.483387
[886]	valid_0's multi_error: 0.483383
[887]	valid_0's multi_error: 0.483366
[888]	valid_0's multi_error: 0.483389
[889]	valid_0's multi_error: 0.483346
[890]	valid_0's multi_error: 0.483338
[891]	valid_0's multi_error: 0.483337
[892]	valid_0's multi_error: 0.483275
[893]	valid_0's multi_error: 0.483319
[894]	valid_0's multi_error: 0.48329
[895]	valid_0

...

[166]	valid_0's multi_error: 0.502125
[167]	valid_0's multi_error: 0.502002
[168]	valid_0's multi_error: 0.501904
[169]	valid_0's multi_error: 0.501904
[170]	valid_0's multi_error: 0.501793
[171]	valid_0's multi_error: 0.501651
[172]	valid_0's multi_error: 0.501475
[173]	valid_0's multi_error: 0.501444
[174]	valid_0's multi_error: 0.501372
[175]	valid_0's multi_error: 0.501329
[176]	valid_0's multi_error: 0.501156
[177]	valid_0's multi_error: 0.501138
[178]	valid_0's multi_error: 0.500996
[179]	valid_0's multi_error: 0.500856
[180]	valid_0's multi_error: 0.500841
[181]	valid_0's multi_error: 0.500701
[182]	valid_0's multi_error: 0.500632
[183]	valid_0's multi_error: 0.500593
[184]	valid_0's multi_error: 0.500484
[185]	valid_0's multi_error: 0.500483
[186]	valid_0's multi_error: 0.50036
[187]	valid_0's multi_error: 0.500289
[188]	valid_0's multi_error: 0.500238
[189]	valid_0's multi_error: 0.50011
[190]	valid_0's multi_error: 0.499949
[191]	valid_0's multi_error: 0.499941
[192]	valid_0'

[383]	valid_0's multi_error: 0.489765
[384]	valid_0's multi_error: 0.489699
[385]	valid_0's multi_error: 0.489723
[386]	valid_0's multi_error: 0.489653
[387]	valid_0's multi_error: 0.489619
[388]	valid_0's multi_error: 0.489578
[389]	valid_0's multi_error: 0.48956
[390]	valid_0's multi_error: 0.489471
[391]	valid_0's multi_error: 0.489486
[392]	valid_0's multi_error: 0.489478
[393]	valid_0's multi_error: 0.489416
[394]	valid_0's multi_error: 0.489428
[395]	valid_0's multi_error: 0.489414
[396]	valid_0's multi_error: 0.489392
[397]	valid_0's multi_error: 0.489298
[398]	valid_0's multi_error: 0.489248
[399]	valid_0's multi_error: 0.489259
[400]	valid_0's multi_error: 0.489274
[401]	valid_0's multi_error: 0.489222
[402]	valid_0's multi_error: 0.489195
[403]	valid_0's multi_error: 0.489183
[404]	valid_0's multi_error: 0.489137
[405]	valid_0's multi_error: 0.489048
[406]	valid_0's multi_error: 0.489054
[407]	valid_0's multi_error: 0.489062
[408]	valid_0's multi_error: 0.489047
[409]	valid_0

[600]	valid_0's multi_error: 0.485843
[601]	valid_0's multi_error: 0.485822
[602]	valid_0's multi_error: 0.485796
[603]	valid_0's multi_error: 0.485765
[604]	valid_0's multi_error: 0.48576
[605]	valid_0's multi_error: 0.485737
[606]	valid_0's multi_error: 0.485746
[607]	valid_0's multi_error: 0.485737
[608]	valid_0's multi_error: 0.485722
[609]	valid_0's multi_error: 0.485701
[610]	valid_0's multi_error: 0.485701
[611]	valid_0's multi_error: 0.485705
[612]	valid_0's multi_error: 0.485657
[613]	valid_0's multi_error: 0.485686
[614]	valid_0's multi_error: 0.485702
[615]	valid_0's multi_error: 0.485698
[616]	valid_0's multi_error: 0.485704
[617]	valid_0's multi_error: 0.485719
[618]	valid_0's multi_error: 0.485675
[619]	valid_0's multi_error: 0.485717
[620]	valid_0's multi_error: 0.485722
[621]	valid_0's multi_error: 0.485725
[622]	valid_0's multi_error: 0.485757
[623]	valid_0's multi_error: 0.485725
[624]	valid_0's multi_error: 0.485716
[625]	valid_0's multi_error: 0.48565
[626]	valid_0'

[817]	valid_0's multi_error: 0.484054
[818]	valid_0's multi_error: 0.484068
[819]	valid_0's multi_error: 0.484047
[820]	valid_0's multi_error: 0.484051
[821]	valid_0's multi_error: 0.484035
[822]	valid_0's multi_error: 0.484035
[823]	valid_0's multi_error: 0.484034
[824]	valid_0's multi_error: 0.484008
[825]	valid_0's multi_error: 0.483974
[826]	valid_0's multi_error: 0.48398
[827]	valid_0's multi_error: 0.483968
[828]	valid_0's multi_error: 0.483928
[829]	valid_0's multi_error: 0.483917
[830]	valid_0's multi_error: 0.483935
[831]	valid_0's multi_error: 0.48392
[832]	valid_0's multi_error: 0.483948
[833]	valid_0's multi_error: 0.483945
[834]	valid_0's multi_error: 0.483935
[835]	valid_0's multi_error: 0.483941
[836]	valid_0's multi_error: 0.483948
[837]	valid_0's multi_error: 0.483938
[838]	valid_0's multi_error: 0.483941
[839]	valid_0's multi_error: 0.483957
[840]	valid_0's multi_error: 0.483937
[841]	valid_0's multi_error: 0.483945
[842]	valid_0's multi_error: 0.483916
[843]	valid_0'

d:\python27\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [16]:
age_train['age_group'].nunique()

6

In [12]:
history


import numpy as np
import pandas as pd

import lightgbm as lgb
pd.set_option('display.max_columns',None)
#读取数据
age_train = pd.read_csv("age_train.csv", names=['uid','age_group'])
age_test = pd.read_csv("age_test.csv", names=['uid'])
user_basic_info = pd.read_csv("user_basic_info.csv", names=['uid','gender','city','prodName','ramCapacity','ramLeftRation','romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
user_behavior_info = pd.read_csv("user_behavior_info.csv", names=['uid','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum'])
user_app_actived = pd.read_csv("user_app_actived.csv", names=['uid','appId'])
#user_app_usage = pd.read_csv("user_app_usage.csv")
app_info = pd.read_csv("app_info.csv", names=['appId', 'category'])
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def f(x):
    s = x.value_counts()
    return np.nan if len(s) == 0 else s.index[0]
def processUserAppUsage():
    resTable = p